Alumnos por categoria de buen o mal internet, un grafico para cada componetencia-por grado. 

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import athena_utils as athena

In [9]:
projects_id= 72

In [10]:
query_cancelaciones= f'''

WITH
cte_Organization AS (
    SELECT
        poa.project_id,
        ARRAY_JOIN(ARRAY_AGG(DISTINCT o.name), ', ') AS org_names
    FROM datalake.project_organization_association poa
    JOIN datalake.organizations o ON poa.organization_id = o.id
    GROUP BY poa.project_id
),
cte_ProgramType AS (
    SELECT
        ppta.project_id,
        ARRAY_JOIN(ARRAY_AGG(DISTINCT cbpt.name), ', ') AS program_types
    FROM datalake.project_program_type_association ppta
    JOIN datalake.catalog_b2bprogramtype cbpt ON ppta.program_type_id = cbpt.id
    GROUP BY ppta.project_id
),
base AS (
    SELECT
        DISTINCT
        p.id AS projectsID,
        p.name AS "Proyecto",
        COALESCE(p.type, 'B2C') AS "Canal",
        pt.program_types AS "Tipo de programa",
        o.org_names AS "Organización",
        (CASE WHEN ee.institution IS NULL THEN ei.name ELSE ee.institution END) AS institucion,
        (CASE WHEN ee.group_section IS NULL THEN rr.college_group ELSE ee.group_section END) AS seccion,
        ee.career AS career,
        rr.id AS room,
        rr.name AS "Room Name",
        CONCAT('https://backoffice.crackthecode.la/dashboard/rooms/', CAST(rr.id AS VARCHAR)) AS "Link Room",
        rs.id AS sesionID,
        rs.session_number AS sesion,
        ee.grade AS grado,
        (CASE WHEN rs.cancellation_reason_id IS NULL THEN 36 ELSE rs.cancellation_reason_id END) AS reasonID,
        (CASE WHEN rc.name IS NULL THEN 'N/A' ELSE rc.name END) AS Motivo,
        rs.start_date AS "Fecha",
        rs.start_time AS Hora,
        SUBSTR(rs.start_time, 1, 2) AS "Hora Agrupada",
        DATE_TRUNC('week', rs.start_date) AS ordenSemana,
        CONCAT(
            DATE_FORMAT(DATE_TRUNC('week', rs.start_date), '%d/%m'),
            '-',
            DATE_FORMAT(DATE_TRUNC('week', rs.start_date) + INTERVAL '6' DAY, '%d/%m')
        ) AS semana,
        DATE_FORMAT(rs.start_date, '%W') AS dia,
        rs.state AS state,
        rs.risk_cancellation AS Riesgo,
        CONCAT(au.last_name, ', ', au.first_name) AS profesor
    FROM
        datalake.room_roomsessions rs
        LEFT JOIN datalake.room_room rr ON rs.room_id = rr.id
        JOIN datalake.enrollment_enrolment ee ON rr.id = ee.room_id AND ee.b2b_project_id IS NOT NULL
        LEFT JOIN datalake.projects p ON (ee.b2b_project_id = p.id OR p.id = rr.project_b2b_id)
        LEFT JOIN cte_Organization o ON p.id = o.project_id
        LEFT JOIN cte_ProgramType pt ON p.id = pt.project_id
        LEFT JOIN datalake.account_user au ON rr.teacher_id = au.id
        LEFT JOIN datalake.catalog_reasonsessioncancellation rc ON (
            (CASE WHEN (rs.state = 'false' AND rs.cancellation_reason_id IS NULL) THEN 36 ELSE rs.cancellation_reason_id END) = rc.id
        )
        LEFT JOIN datalake.educational_institution ei ON ei.id = rr.educational_institution_id
    ORDER BY
        DATE_TRUNC('week', rs.start_date) ASC
)
SELECT *
FROM base
WHERE
    NOT (state = 'false' AND projectsID IN (19, 14) AND (("Fecha" BETWEEN DATE '2024-06-17' AND DATE '2024-07-05') OR ("Fecha" BETWEEN DATE '2024-10-07' AND DATE '2024-10-14')))
    AND
    NOT (state = 'false' AND projectsID IN (47, 48, 56) AND ("Fecha" BETWEEN DATE '2024-10-07' AND DATE '2024-10-14'))
    AND
    (reasonID NOT IN (33, 34, 35) OR reasonID IS NULL)
    
    and projectsID IN ({projects_id})
    
    ;


'''

In [ ]:
query_asistencias='''

SELECT
*
FROM
asistencia_alumnos

where 
b2b_project_id IN ({projects_id})

'''

In [ ]:
query_cuestionarios = f''' 

SELECT DISTINCT
     me.moodle_id moodle_user_id
   , 'Moodle' origen
   , ss.id student_id
   , ee.institution educative_institution
   , ee.grade grade
   , concat(ee.grade,'+', ee.group_section) grade_section
   , ee.career career
   , ee.educational_level educational_level
   , DATE_DIFF('year', ss.birthdate, p.operative_start_date) age 
   , ss.gender genero
   , ipp.inscriptos_activos activos_por_proyecto
   , ipi.inscriptos_activos activos_por_educative_institution
   , ipg.inscriptos_activos activos_por_grade
   , p.id project_id
   , p.name project_name
   , ce.course_id moodle_course_id
   , rr.id room_id
   , ce.unique_id evaluation_unique_id
   , ce.name evaluation_name
   , ceq.name question_name
   , ceq.tag tag_question
   , ceq.question_id question_id
   , ceq.question_name question
   , cer.answer answer
   , cer.right_answer right_answer
   , ce.tag AS tipo_test

   FROM
   moodle_enrollment me
   LEFT JOIN moodle_course_evaluations ce ON (me.course_id = ce.course_id)
   LEFT JOIN moodle_course_evaluation_questions ceq ON (ce.unique_id = ceq.unique_id) AND ((ceq.question_name <> 'label') OR (ceq.question_name IS NULL))
   LEFT JOIN moodle_course_evaluation_responses cer ON ((cer.unique_id = ceq.unique_id) AND (ceq.question_id = cer.question_id) AND (me.moodle_id = cer.moodle_id) AND (ce.type <> 'assign')  AND (cer.attempt_time_finish IS NOT NULL))
   INNER JOIN room_room rr ON (rr.course_mdl_id = me.course_id)
   LEFT JOIN student_student ss ON (ss.user_mdl_id = me.moodle_id)
   LEFT JOIN room_room_students rrs ON ((rrs.student_id = ss.id) AND (rrs.room_id = rr.id))
   LEFT JOIN enrollment_enrolment ee ON (((ee.group_id = rr.group_id) OR (ee.room_id = rr.id)) AND (ee.student_id = ss.id) AND (ee.state <> 'cancel') AND (ee.state <> 'inactive'))
   LEFT JOIN projects p ON (p.id = ee.b2b_project_id)
   
   WHERE (p.id in ({projects_id}) and (me.role = 'student'))



'''

In [12]:
df_cancelaciones=athena.run_athena_query_auto(query_cancelaciones)
df_asistencias=athena.run_athena_query_auto(query_asistencias)

In [13]:
df_asistencias

,student_id_1,number_of_session,numero_sesion_bo,sesion_numero,estado_asistencia,estado_asistencia_simple,flag_asistencia,flag_backoffice_attendance,asistencia_estudiante,course_id,...,room_name,flag_baja,institution,grade,group_section,flag_hackathon,career,ciudad,promedio_de_asistencia,flag_ultima_sesion
0,27864,3,3,Sesión 3,Vacio/Indefinido,Vacio/Indefinido,0,0,0.01,503,...,Educación +Innovación 2025 - E.N.S. La Haciend...,Activo,E.N.S. La Hacienda,9,B,No Hackathon,None,None,0.941176,None
1,27877,4,6,Sesión 4,A tiempo,A tiempo,1,1,1.00,503,...,Educación +Innovación 2025 - E.N.S. La Haciend...,Activo,E.N.S. La Hacienda,9,B,No Hackathon,None,None,0.882353,None
2,27915,4,6,Sesión 4,A tiempo,A tiempo,1,1,1.00,503,...,Educación +Innovación 2025 - E.N.S. La Haciend...,Activo,E.N.S. La Hacienda,9,B,No Hackathon,None,None,0.882353,None
3,27883,5,7,Sesión 5,Falta,Falta,0,3,0.00,503,...,Educación +Innovación 2025 - E.N.S. La Haciend...,Activo,E.N.S. La Hacienda,9,B,No Hackathon,None,None,0.882353,None
4,27877,6,8,Sesión 6,A tiempo,A tiempo,1,1,1.00,503,...,Educación +Innovación 2025 - E.N.S. La Haciend...,Activo,E.N.S. La Hacienda,9,B,No Hackathon,None,None,0.882353,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423827,30976,17,27,Sesión 17,A tiempo,A tiempo,1,1,1.00,525,...,Estud-IA - I.E. San José Obrero - 9C,Activo,I.E. San José Obrero,9,C,No Hackathon,None,None,0.722222,None
423828,31436,17,27,Sesión 17,A tiempo,A tiempo,1,1,1.00,525,...,Estud-IA - I.E. San José Obrero - 9C,Activo,I.E. San José Obrero,9,C,No Hackathon,None,None,1.000000,None
423829,31436,18,32,Sesión 18,A tiempo,A tiempo,1,1,1.00,525,...,Estud-IA - I.E. San José Obrero - 9C,Activo,I.E. San José Obrero,9,C,No Hackathon,None,None,1.000000,Ultima Sesion
423830,30970,18,32,Sesión 18,A tiempo,A tiempo,1,1,1.00,525,...,Estud-IA - I.E. San José Obrero - 9C,Activo,I.E. San José Obrero,9,C,No Hackathon,None,None,0.722222,Ultima Sesion
